In [35]:
import numpy as np
from rpy2.robjects import pandas2ri, r
from rpy2.robjects.packages import importr
pandas2ri.activate()
import pandas as pd
# Load qgenet package in R
qgenet = importr('qgenet')

# Define your pedigree data
ped_data = {
    'ID': [1, 2, 3, 4, 5, 6, 7, 8],
    'SIRE': [0, 0, 0, 1, 3, 1, 4, 3],
    'DAM': [0, 0, 0, 0, 2, 2, 5, 6]
}

# Convert your pedigree data to an R dataframe
ped_df = pandas2ri.py2rpy_pandasdataframe(pd.DataFrame(ped_data))

# Compute the A matrix using qgenet package in R
A = qgenet.amatrix(ped_df)

# Invert the A matrix
#ainv = np.array(r.solve(A))
ainv = np.linalg.inv(A)
print(np.array(A))
#np.allclose(ainv,e)

[[1.    0.    0.    0.5   0.    0.5   0.25  0.25 ]
 [0.    1.    0.    0.    0.5   0.5   0.25  0.25 ]
 [0.    0.    1.    0.    0.5   0.    0.25  0.5  ]
 [0.5   0.    0.    1.    0.    0.25  0.5   0.125]
 [0.    0.5   0.5   0.    1.    0.25  0.5   0.375]
 [0.5   0.5   0.    0.25  0.25  1.    0.25  0.5  ]
 [0.25  0.25  0.25  0.5   0.5   0.25  1.    0.25 ]
 [0.25  0.25  0.5   0.125 0.375 0.5   0.25  1.   ]]


In [25]:
# Example fixed effects (mean) and random effects (animal genetic effects) in Python
X = np.ones((5,1))  # Assuming 5 observations and only a mean effect
Z = np.array([
    [0, 0, 0, 1, 0, 0, 0, 0],  # Observation for individual 4
    [0, 0, 0, 0, 1, 0, 0, 0],  # Observation for individual 5
    [0, 0, 0, 0, 0, 1, 0, 0],  # Observation for individual 6
    [0, 0, 0, 0, 0, 0, 1, 0],  # Observation for individual 7
    [0, 0, 0, 0, 0, 0, 0, 1]   # Observation for individual 8
])
# Observations
y = np.array([4.5, 2.9, 3.9, 3.5, 5.0]).reshape(-1, 1)

# Matrix operations
XX = X.T @ X
XZ = X.T @ Z
ZX = Z.T @ X
ZZ = Z.T @ Z
ZZA = ZZ + ainv * 2  # Adjust based on your specific model
Xy = X.T @ y
Zy = Z.T @ y

# Assemble mixed model equations
mme = np.vstack([
    np.hstack([XX, XZ]),
    np.hstack([ZX, ZZA])
])

# Solution vector
sol = np.linalg.solve(mme, np.vstack([Xy, Zy]))

print("Solution:", sol)


Solution: [[ 3.97256482]
 [ 0.12596923]
 [-0.15757814]
 [-0.024685  ]
 [ 0.14742549]
 [-0.31838503]
 [ 0.05387085]
 [-0.16289678]
 [ 0.21716138]]


In [30]:
import gblup
# Number of observations
n_obs = 5

# IDs of individuals with observations (assuming these are 1-indexed and correspond to the order in 'ainv')
obs_ids = [3,4,5,6,7]

# Phenotype values for these observations
y_values = [4.5, 2.9, 3.9, 3.5, 5.0]


# Solve the mixed model
solution_vector = gblup.solve_mixed_gblup_model(n_obs, obs_ids, y_values, A)

print("Solution vector:", solution_vector)

Solution vector: [[ 3.97256482]
 [ 0.12596923]
 [-0.15757814]
 [-0.024685  ]
 [ 0.14742549]
 [-0.31838503]
 [ 0.05387085]
 [-0.16289678]
 [ 0.21716138]]


In [31]:
np.allclose(solution_vector, sol)

True

In [32]:
import numpy as np

def generate_synthetic_data(n_individuals, n_observations):
    """
    Generate synthetic pedigree, observation IDs, and phenotype values.
    
    Parameters:
    - n_individuals: Number of individuals in the pedigree.
    - n_observations: Number of observations (must be <= n_individuals).
    
    Returns:
    - ainv: Synthetic additive relationship matrix inverse for the pedigree.
    - obs_ids: IDs of individuals with observations.
    - y_values: Synthetic phenotype values for observations.
    """
    # Generate a synthetic additive relationship matrix (A matrix) and invert it
    # Here, we'll use an identity matrix for simplicity; in practice, A would be based on pedigree
    A = np.eye(n_individuals)
    ainv = np.linalg.inv(A + np.random.normal(0, 0.05, A.shape))  # Adding noise for a more realistic scenario
    
    # Generate observation IDs (random sample of individuals)
    obs_ids = np.random.choice(range(0, n_individuals), n_observations, replace=False)
    
    # Generate synthetic phenotype values (normal distribution)
    y_values = np.random.normal(10, 2, n_observations)
    
    return ainv, obs_ids, y_values

# Parameters for synthetic data generation
n_individuals = 6000  # Number of individuals in the pedigree
n_observations = 3000  # Number of observations

# Generate synthetic data
ainv, obs_ids, y_values = generate_synthetic_data(n_individuals, n_observations)


In [33]:
solution_vector = gblup.solve_mixed_gblup_model(n_observations, obs_ids, y_values, ainv)
solution_vector

array([[  9.65855164],
       [ 26.53936216],
       [ -3.45564058],
       ...,
       [ -1.37455656],
       [ 14.0913003 ],
       [-18.79276052]])